In [2]:
import geopandas as gpd
import os
from datetime import datetime
from shapely.geometry import LineString
    
# Read the shapefile
gdf = gpd.read_file(r"C:\Users\Logan\Desktop\School Files\GIS 5572\Final\BaseLines\no_traffic_lines.shp")

# Calculate the centroid coordinates
gdf['Latitude'] = gdf['geometry'].centroid.y
gdf['Longitude'] = gdf['geometry'].centroid.x
gdf['geometry_wkt'] = gdf['geometry'].to_wkt()
gdf['SegmentLength'] = gdf['FromCumu_2'] - gdf['ToCumul_Ki']
gdf['Distance'] = (gdf['FromCumu_2'] + gdf['ToCumul_Ki']) / 2

# Select and rename columns
columns_to_keep = {
    'SourceOID': 'SourceOID',
    'Latitude': 'Latitude',
    'Longitude': 'Longitude',
    'geometry_wkt' : 'geometry_wkt',
    'SegmentLength' : 'SegmentLength',
    'Distance' : 'Distance'
}
gdf = gdf.rename(columns=columns_to_keep)[list(columns_to_keep.values())]

# Save to CSV
gdf.to_csv(r"C:\Users\Logan\Desktop\School Files\GIS 5572\Final\ML_Base_data_new.csv", index=False)
print(f"Saved")

C:\Users\Logan\AppData\Local\Temp\ipykernel_12272\2895185673.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['Latitude'] = gdf['geometry'].centroid.y
C:\Users\Logan\AppData\Local\Temp\ipykernel_12272\2895185673.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['Longitude'] = gdf['geometry'].centroid.x


Saved


In [ ]:
import os
import pandas as pd
# Load the data from the CSV file into a DataFrame
data = pd.read_csv(r"C:\Users\Logan\Desktop\School Files\GIS 5572\Final\AllTrafficCost3.csv")

# Create a random sample of 20% of the data
sampled_data = data.sample(frac=0.2, random_state=42)

# Save the sampled data to a new CSV file
sampled_data.to_csv('SampledData2.csv', index=False)